In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import re

In [2]:
pd.set_option('display.max_colwidth',500)
pd.set_option('display.max_rows',500)

In [3]:
play = pd.read_csv("play_by_play.csv")

In [4]:
play = play.drop(['eventnum', 'eventmsgactiontype', 'pctimestring', 'neutraldescription','score', 'person1type', 
                  'player1_id', 'player1_name', 'player1_team_abbreviation', 'person2type', 'player2_id', 'player2_name', 
                  'player2_team_abbreviation', 'person3type', 'player3_id', 'player3_name', 'player3_team_abbreviation', 
                  'video_available_flag'], axis = 1)
play

,game_id,eventmsgtype,period,wctimestring,homedescription,visitordescription,scoremargin,player1_team_id,player1_team_city,player1_team_nickname,player2_team_id,player2_team_city,player2_team_nickname,player3_team_id,player3_team_city,player3_team_nickname
0,29600009,12,1,11:44 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29600009,10,1,11:45 PM,Jump Ball Olajuwon vs. Polynice: Tip to Williamson,NaN,NaN,1.610613e+09,Houston,Rockets,1.610613e+09,Sacramento,Kings,1.610613e+09,Sacramento,Kings
2,29600009,5,1,11:46 PM,NaN,Williamson Traveling Turnover (P1.T1),NaN,1.610613e+09,Sacramento,Kings,NaN,NaN,NaN,NaN,NaN,NaN
3,29600009,2,1,11:46 PM,MISS Olajuwon 13' Jump Shot,NaN,NaN,1.610613e+09,Houston,Rockets,NaN,NaN,NaN,NaN,NaN,NaN
4,29600009,4,1,11:46 PM,Drexler REBOUND (Off:1 Def:0),NaN,NaN,1.610613e+09,Houston,Rockets,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13227006,22201014,8,4,6:01 PM,SUB: Porter Jr. FOR Smith,NaN,NaN,1.610613e+09,Denver,Nuggets,1.610613e+09,Denver,Nuggets,NaN,NaN,NaN
13227007,22201014,8,4,6:01 PM,NaN,SUB: Claxton FOR Johnson,NaN,1.610613e+09,Brooklyn,Nets,1.610613e+09,Brooklyn,Nets,NaN,NaN,NaN
13227008,22201014,2,4,6:02 PM,MISS Jokic 25' 3PT Turnaround Jump Shot,NaN,NaN,1.610613e+09,Denver,Nuggets,NaN,NaN,NaN,NaN,NaN,NaN
13227009,22201014,4,4,6:02 PM,NUGGETS Rebound,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
game = pd.read_csv("game_summary.csv")

In [6]:
game_dict = {'1946' : [], '1947' : [], '1948' : [], '1949' : [], '1950' : [], '1951' : [], '1952' : [], '1953' : [], '1954' : [], 
             '1955' : [],'1956' : [], '1957' : [], '1958' : [], '1959' : [], '1960' : [], '1961' : [], '1962' : [], '1963' : [], 
             '1964' : [], '1965' : [], '1966' : [], '1967' : [], '1968' : [],'1969' : [], '1970' : [], '1971' : [], '1972' : [], 
             '1973' : [], '1974' : [], '1975' : [], '1976' : [], '1977' : [], '1978' : [], '1979' : [], '1980' : [], '1981' : [],
             '1982' : [], '1983' : [], '1984' : [], '1985' : [], '1986' : [], '1987' : [], '1988' : [], '1989' : [], '1990' : [], 
             '1991' : [], '1992' : [], '1993' : [], '1994' : [],'1995' : [], '1996' : [], '1997' : [], '1998' : [], '1999' : [], 
             '2000' : [], '2001' : [], '2002' : [], '2003' : [], '2004' : [], '2005' : [], '2006' : [], '2007' : [],'2008' : [], 
             '2009' : [], '2010' : [], '2011' : [], '2012' : [], '2013' : [], '2014' : [], '2015' : [], '2016' : [], '2017' : [], 
             '2018' : [], '2019' : [], '2020' : [],'2021' : [], '2022' : [], '2023' : []}
for row in game.iterrows():
    season = str(row[1]["season"])
    game_dict[season].append(row[1]["game_id"])

In [7]:
player = pd.DataFrame(data = {"season" : [], 'team_id' : [], "team_city" : [], 'team_nickname' : [], 'field_goals_made' : [], 
                              'field_goals_ast' : [], 'field_goals_missed' : [], 'times_ball_lost' : [], 'times_ball_stolen' : [], 
                              'fouls' : [], 'game_id' : [], 'games_played' : [], 'field_goals_blocked' : [], 'free_throws_made' : [], 
                              'free_throws_missed' : [], "points" : [], "three_pointers" : [], 'rebounds' : [],
                              "ORB" : [], "DRB" : []})
player

,season,team_id,team_city,team_nickname,field_goals_made,field_goals_ast,field_goals_missed,times_ball_lost,times_ball_stolen,fouls,game_id,games_played,field_goals_blocked,free_throws_made,free_throws_missed,points,three_pointers,rebounds,ORB,DRB


In [8]:
game_id = 0
count = 1
margin = 0
game_id_index = 0
eventmsgtype = 1
homedescription = 4
visitordescription = 5
scoremargin = 6
player1_team_id = 7
player1_team_city = 8
player1_team_nickname = 9
player2_team_id = 10
player2_team_city = 11
player2_team_nickname = 12
player3_team_id = 13
player3_team_city = 14
player3_team_nickname = 15
pattern = re.compile("\d+")
print("starting...")
for key, values in game_dict.items():
    key = int(key)
    if key < 1996:
        continue
    print(key, end = ", ")
    for game in values:
        for row in play.loc[play["game_id"] == game].values:
            data = {"season" : key, 'team_id' : 0, "team_city" : 0, 'team_nickname' : 0, 'field_goals_made' : 0, 'field_goals_ast' : 0, 
                    'field_goals_missed' : 0, 'times_ball_lost' : 0, 'times_ball_stolen' : 0, 'fouls' : 0, 'game_id' : 0, 
                    'games_played' : 0, 'field_goals_blocked' : 0, 'free_throws_made' : 0, 'free_throws_missed' : 0, 
                    "points" : 0, "three_pointers" : 0, 'rebounds' : np.nan, "start_time" : 0, "MP" : 0, "ORB" : 0, "DRB" : 0}
            if (
               row[eventmsgtype] == 7 or row[eventmsgtype] == 8 or row[eventmsgtype] == 7 or row[eventmsgtype] == 9 or row[eventmsgtype] == 10 or
               row[eventmsgtype] == 11 or row[eventmsgtype] == 12 or row[eventmsgtype] == 13 or pd.isna(row[player1_team_id])
               ):
                    continue

            elif row[eventmsgtype] == 1:
                if not player.loc[((player["season"] == key) & (player["team_id"] == row[player1_team_id])), "team_id"].empty:
                    player.loc[(player["team_id"] == row[player1_team_id]) & (player["season"] == key), "field_goals_made"] += 1
                    if row[scoremargin] == "TIE":
                        points = abs(0 - margin)
                    else:
                        points = abs(int(row[scoremargin]) - margin)
                    player.loc[(player["team_id"] == row[player1_team_id]) & (player["season"] == key), "points"] += points
                    if points == 3:
                        player.loc[(player["team_id"] == row[player1_team_id]) & (player["season"] == key), "three_pointers"] += 1
                    if row[scoremargin] == "TIE":
                        margin = 0
                    else:
                        margin = int(row[scoremargin])
                else:
                    data["team_id"] = row[player1_team_id]
                    data["team_nickname"] = row[player1_team_nickname]
                    data["team_city"] = row[player1_team_city]
                    data["field_goals_made"] = 1
                    player = player.append(data, ignore_index=True)

                    if row[scoremargin] == "TIE":
                        points = abs(0 - margin)
                    else:
                        points = abs(int(row[scoremargin]) - margin)
                    player.loc[(player["team_id"] == row[player1_team_id]) & (player["season"] == key), "points"] += points
                    if points == 3:
                        player.loc[(player["team_id"] == row[player1_team_id]) & (player["season"] == key), "three_pointers"] += 1
                    if row[scoremargin] == "TIE":
                        margin = 0
                    else:
                        margin = int(row[scoremargin])

                if pd.notna(row[player2_team_id]):
                    if not player.loc[((player["season"] == key) & (player["team_id"] == row[player2_team_id])), "team_id"].empty:
                        player.loc[(player["team_id"] == row[player2_team_id]) & (player["season"] == key), 'field_goals_ast'] += 1
                    else:
                        data["team_id"] = row[player2_team_id]
                        data["team_nickname"] = row[player2_team_id]
                        data["team_city"] = row[player2_team_city]
                        data["field_goals_ast"] = 1
                        player = player.append(data, ignore_index=True)


            elif row[eventmsgtype] == 2:
                if not player.loc[((player["season"] == key) & (player["team_id"] == row[player1_team_id])), "team_id"].empty:
                    player.loc[(player["team_id"] == row[player1_team_id]) & (player["season"] == key), "field_goals_missed"] += 1
                else:
                    data["team_id"] = row[player1_team_id]
                    data["team_nickname"] = row[player1_team_nickname]
                    data["team_city"] = row[player1_team_city]
                    data["field_goals_missed"] = 1
                    player = player.append(data, ignore_index=True)


                if pd.notna(row[player3_team_id]):
                    if not player.loc[((player["season"] == key) & (player["team_id"] == row[player3_team_id])), "team_id"].empty:
                        player.loc[(player["team_id"] == row[player3_team_id]) & (player["season"] == key), "field_goals_blocked"] += 1
                    else:
                        data["team_id"] = row[player3_team_id]
                        data["team_nickname"] = row[player3_team_nickname]
                        data["team_city"] = row[player3_team_city]
                        data["field_goals_blocked"] = 1
                        player = player.append(data, ignore_index=True)

                        
            elif row[eventmsgtype] == 3:
                if pd.notna(row[homedescription]):
                    if "MISS" in row[homedescription]:
                        if not player.loc[(player["team_id"] == row[player1_team_id]) & (player["season"] == key), "team_id"].empty:
                            player.loc[(player["team_id"] == row[player1_team_id]) & (player["season"] == key), "free_throws_missed"] += 1
                        else:
                            data["team_id"] = row[player1_team_id]
                            data["team_nickname"] = row[player1_team_nickname]
                            data["team_city"] = row[player1_team_city]
                            data["free_throws_missed"] = 1
                            player = player.append(data, ignore_index=True)
                    else:
                        if not player.loc[(player["team_id"] == row[player1_team_id]) & (player["season"] == key), "team_id"].empty:
                            player.loc[(player["team_id"] == row[player1_team_id]) & (player["season"] == key), "free_throws_made"] += 1
                        else:
                            data["team_id"] = row[player1_team_id]
                            data["team_nickname"] = row[player1_team_nickname]
                            data["team_city"] = row[player1_team_city]
                            data["free_throws_made"] = 1
                            player = player.append(data, ignore_index=True)

                else:
                    if "MISS" in row[visitordescription]:
                        if not player.loc[(player["team_id"] == row[player1_team_id]) & (player["season"] == key), "team_id"].empty:
                            player.loc[(player["team_id"] == row[player1_team_id]) & (player["season"] == key), "free_throws_missed"] += 1
                        else:
                            data["team_id"] = row[player1_team_id]
                            data["team_nickname"] = row[player1_team_nickname]
                            data["team_city"] = row[player1_team_city]
                            data["free_throws_missed"] = 1
                            player = player.append(data, ignore_index=True)

                    else:
                        if not player.loc[(player["team_id"] == row[player1_team_id]) & (player["season"] == key), "team_id"].empty:
                            player.loc[(player["team_id"] == row[player1_team_id]) & (player["season"] == key), "free_throws_made"] += 1
                        else:
                            data["team_id"] = row[player1_team_id]
                            data["team_nickname"] = row[player1_team_nickname]
                            data["team_city"] = row[player1_team_city]
                            data["free_throws_made"] = 1
                            player = player.append(data, ignore_index=True)
            
            elif row[eventmsgtype] == 4:
                if pd.notna(row[homedescription]):
                    if not player.loc[(player["team_id"] == row[player1_team_id]) & (player["season"] == key), "team_id"].empty:
                        player.loc[(player["team_id"] == row[player1_team_id]) & (player["season"] == key), "rebounds"] = row[homedescription]
                    else:
                        data["team_nickname"] = row[player1_team_nickname]
                        data["team_id"] = row[player1_team_id]
                        data["rebounds"] = row[homedescription]
                        player = player.append(data, ignore_index=True)
                else:
                    if not player.loc[(player["team_id"] == row[player1_team_id]) & (player["season"] == key), "team_id"].empty:
                        player.loc[(player["team_id"] == row[player1_team_id]) & (player["season"] == key), "rebounds"] = row[visitordescription]
                    else:
                        data["team_nickname"] = row[player1_team_nickname]
                        data["team_id"] = row[player1_team_id]
                        data["rebounds"] = row[visitordescription]
                        player = player.append(data, ignore_index=True)
                        
            elif row[eventmsgtype] == 5:
                if not player.loc[((player["season"] == key) & (player["team_id"] == row[player1_team_id])), "team_id"].empty:
                    player.loc[(player["team_id"] == row[player1_team_id]) & (player["season"] == key), "times_ball_lost"] += 1
                else:
                    data["team_id"] = row[player1_team_id]
                    data["team_nickname"] = row[player1_team_nickname]
                    data["team_city"] = row[player1_team_city]
                    data["times_ball_lost"] = 1
                    player = player.append(data, ignore_index=True)
                    
                if pd.notna(row[player2_team_id]):
                    if not player.loc[((player["season"] == key) & (player["team_id"] == row[player2_team_id])), "team_id"].empty:
                        player.loc[(player["team_id"] == row[player2_team_id]) & (player["season"] == key), "times_ball_stolen"] += 1
                    else:
                        data["team_id"] = row[player2_team_id]
                        data["team_nickname"] = row[player2_team_nickname]
                        data["team_city"] = row[player2_team_city]
                        data["times_ball_stolen"] = 1
                        player = player.append(data, ignore_index=True)

            elif row[eventmsgtype] == 6:
                if not player.loc[((player["season"] == key) & (player["team_id"] == row[player1_team_id])), "team_id"].empty:
                    player.loc[(player["team_id"] == row[player1_team_id]) & (player["season"] == key), "fouls"] += 1
                else:
                    data["team_id"] = row[player1_team_id]
                    data["team_city"] = row[player1_team_city]
                    data["team_nickname"] = row[player1_team_nickname]
                    data["fouls"] = 1
                    player = player.append(data, ignore_index=True)
            
            game_id = row[game_id_index]
            
            if True in ((player.loc[(player["team_id"] == row[player1_team_id]) & (player["season"] == key)]).values.tolist()):
                if game_id != (player.loc[(player["team_id"] == row[player1_team_id]) & (player["season"] == key), "game_id"].values[0]):
                    player.loc[(player["team_id"] == row[player1_team_id]) & (player["season"] == key), "games_played"] += 1
                    player.loc[(player["team_id"] == row[player1_team_id]) & (player["season"] == key), "game_id"] = game_id
            if True in ((player.loc[(player["team_id"] == row[player2_team_id]) & (player["season"] == key)]).values.tolist()) and row[player2_team_id] != 0:
                if game_id != (player.loc[(player["team_id"] == row[player2_team_id]) & (player["season"] == key), "game_id"].values[0]):
                    player.loc[(player["team_id"] == row[player2_team_id]) & (player["season"] == key), "games_played"] += 1
                    player.loc[(player["team_id"] == row[player2_team_id]) & (player["season"] == key), "game_id"] = game_id
                
        for values in player.loc[player["rebounds"].notna()].values:
            temp = pattern.findall(values[17])
            if len(temp) == 2:
                player.loc[(player["team_id"] == values[1]) & (player["season"] == key), "ORB"] += int(temp[0])
                player.loc[(player["team_id"] == values[1]) & (player["season"] == key), "DRB"] += int(temp[1])
        if count % 10000 == 0:
            print(count, end = ", ")
        count += 1
            
print("Finished")

starting...
1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 10000, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 20000, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, Finished


In [9]:
player

,season,team_id,team_city,team_nickname,field_goals_made,field_goals_ast,field_goals_missed,times_ball_lost,times_ball_stolen,fouls,...,field_goals_blocked,free_throws_made,free_throws_missed,points,three_pointers,rebounds,ORB,DRB,MP,start_time
0,1996.0,1.610613e+09,Sacramento,Kings,2865.0,1715.0,3441.0,1224.0,561.0,1859.0,...,349.0,1402.0,538.0,6231.0,402.0,Smith REBOUND (Off:2 Def:4),2796.0,6289.0,0.0,0.0
1,1996.0,1.610613e+09,Houston,Rockets,2920.0,1927.0,3317.0,1268.0,658.0,1610.0,...,339.0,1453.0,470.0,6500.0,624.0,Barkley REBOUND (Off:3 Def:11),2603.0,6573.0,0.0,0.0
2,1996.0,1.610613e+09,Atlanta,Hawks,2657.0,1464.0,3306.0,1105.0,651.0,1515.0,...,399.0,1414.0,429.0,6195.0,583.0,Barry REBOUND (Off:1 Def:1),2200.0,5121.0,0.0,0.0
3,1996.0,1.610613e+09,Miami,Heat,2654.0,1637.0,3180.0,1182.0,610.0,1802.0,...,414.0,1385.0,535.0,6260.0,567.0,Anderson REBOUND (Off:3 Def:3),2038.0,4812.0,0.0,0.0
4,1996.0,1.610613e+09,Washington,Bullets,2967.0,1775.0,3206.0,1169.0,665.0,1704.0,...,372.0,1297.0,546.0,6285.0,331.0,Webber REBOUND (Off:5 Def:12),1961.0,4595.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
767,2022.0,1.610613e+09,Chicago,Bulls,2614.0,1507.0,2733.0,821.0,467.0,1229.0,...,276.0,1128.0,268.0,6084.0,570.0,Vucevic REBOUND (Off:2 Def:10),54840.0,140562.0,0.0,0.0
768,2022.0,1.610613e+09,Orlando,Magic,2621.0,1475.0,2965.0,939.0,461.0,1363.0,...,304.0,1307.0,352.0,6134.0,668.0,Suggs REBOUND (Off:3 Def:2),33281.0,93764.0,0.0,0.0
769,2022.0,1.610613e+09,Detroit,Pistons,2621.0,1512.0,3141.0,959.0,471.0,1504.0,...,260.0,1350.0,407.0,5994.0,680.0,Livers REBOUND (Off:1 Def:3),37561.0,108198.0,0.0,0.0
770,2022.0,1.610613e+09,LA,Clippers,2619.0,1501.0,2958.0,884.0,454.0,1330.0,...,288.0,1216.0,340.0,6122.0,738.0,Plumlee REBOUND (Off:2 Def:2),37979.0,117447.0,0.0,0.0


In [10]:
from pathlib import Path  
filepath = Path('Lumiere_Reasearch/team_data.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
player.to_csv(filepath)  